In [1]:
from pyspark.sql import SparkSession
from operator import add

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("LingkaiZhu")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/16 18:15:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/16 18:15:19 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


# Part A - Working with the RDD API

A.1.1 Read the English transcripts with Spark, and count the number of lines.

In [20]:
lines_english = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.en")
print(lines_english.first())
lines_english1 = lines_english.map(lambda line: line.split('/n'))
line_english_counts = lines_english1.map(lambda w: len(w))
total_english_lines = line_english_counts.reduce(add)
print(f'total number of lines = {total_english_lines}')

Resumption of the session


total number of lines = 1920225


A.1.2 Do the same with the other language (so that you have a separate lineage of RDDs for each).

In [17]:
lines_sv = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.de")
print(lines_sv.first())
lines_sv1 = lines_sv.map(lambda line: line.split('/n'))
line_sv_counts = lines_sv1.map(lambda w: len(w))
total_sv_lines = line_sv_counts.reduce(add)
print(f'total number of lines = {total_sv_lines}')

Wiederaufnahme der Sitzungsperiode


total number of lines = 1920225


A.1.3 Verify that the line counts are the same for the two languages.
In this case, the count of the english transcripts is 1920225, which is equal to its original language's text.

In [ ]:
A.1.4 Count the number of partitions.

In [26]:
print("number of partitions of the english:", lines_english.getNumPartitions())
print("number of partitions of the original:", lines_sv.getNumPartitions())

number of partitions of the english: 3
number of partitions of the original: 3
